<a href="https://colab.research.google.com/github/ashleyak7/MSCI-151_ashley/blob/main/CW5_Koesnadi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pulp as pl
from collections import defaultdict
import os
import csv

In [12]:
baristas=["Max","Jiwa", "Fore","Donna","Paul"]
days=list(range(1,7))
blocks=list(range(1,5))
H=4
cost={"Max":50000,"Jiwa":50000,"Fore":50000,"Donna":150000,"Paul":150000}
etype={"Max":"P","Jiwa":"P","Fore":"P","Donna":"F","Paul":"F"}
minWeekly_default={"Max":12,"Jiwa":12,"Fore":12, "Donna":36, "Paul":36}

In [14]:
avail = {
    ("Max", 1): 8, ("Max", 2): 8, ("Max", 3): 8, ("Max", 4): 0, ("Max", 5): 8, ("Max", 6): 4, ("Max", 7): 4,
    ("Jiwa", 1): 4, ("Jiwa", 2): 4, ("Jiwa", 3): 8, ("Jiwa", 4): 8, ("Jiwa", 5): 0, ("Jiwa", 6): 4, ("Jiwa", 7): 0,
    ("Fore", 1): 8, ("Fore", 2): 0, ("Fore", 3): 8, ("Fore", 4): 8, ("Fore", 5): 8, ("Fore", 6): 8, ("Fore", 7): 0,
    ("Donna", 1): 12, ("Donna", 2): 12, ("Donna", 3): 12, ("Donna", 4): 12, ("Donna", 5): 12, ("Donna", 6): 12, ("Donna", 7): 8,
    ("Paul", 1): 12, ("Paul", 2): 8, ("Paul", 3): 12, ("Paul", 4): 12, ("Paul", 5): 8, ("Paul", 6): 12, ("Paul", 7): 12
}

In [16]:
days = {
    1: "Monday",
    2: "Tuesday",
    3: "Wednesday",
    4: "Thursday",
    5: "Friday",
    6: "Saturday",
    7: "Sunday"
}

In [17]:
Block_Name={1: "07:00-11:00", 2: "11:00-15:00", 3: "15:00-19:00", 4: "19:00-23:00"}

In [20]:
# Create the problem
prob = pl.LpProblem("Barista_Scheduling", pl.LpMinimize)

# Define the set of days to iterate over (1 to 7)
days_indices = list(days.keys()) # days is now a dict {1:'Monday', ...}

# Decision Variables
# x[(barista, day_index, block)] is 1 if barista works block k on day d, 0 otherwise
x = pl.LpVariable.dicts("Shift",
                        ((b, d_idx, k) for b in baristas for d_idx in days_indices for k in blocks),
                        0, 1, pl.LpBinary)

# Objective Function: Minimize total hours worked
prob += pl.lpSum(x[(b, d_idx, k)] * H
                 for b in baristas for d_idx in days_indices for k in blocks), "Total Hours Worked"

# Constraints

# 1. Each block on each day must be covered by at least one barista
for d_idx in days_indices:
    for k in blocks:
        prob += pl.lpSum(x[(b, d_idx, k)] for b in baristas) >= 1, f"Cover_Block_{d_idx}_{k}"

# 2. Barista daily availability constraint: total hours worked by a barista on a day cannot exceed their available hours
for b in baristas:
    for d_idx in days_indices:
        prob += pl.lpSum(x[(b, d_idx, k)] * H for k in blocks) <= avail[(b, d_idx)], f"Daily_Availability_{b}_{d_idx}"

# 3. Minimum Weekly Hours for each barista
for b in baristas:
    prob += pl.lpSum(x[(b, d_idx, k)] * H
                     for d_idx in days_indices for k in blocks) >= minWeekly_default[b], f"Min_Weekly_Hours_{b}"

# Solve the problem
prob.solve()

# Print the solution status
print("Status:", pl.LpStatus[prob.status])

# Print the schedule
if prob.status == 1: # Changed from pl.LpStatus.Optimal to 1 to avoid AttributeError
    print("\nOptimal Schedule:")
    schedule = defaultdict(lambda: defaultdict(list))
    for v in prob.variables():
        if v.varValue > 0.5: # If the variable is active
            if v.name.startswith("Shift_"):
                # Extract barista, day, block from variable name
                # Variable names are like Shift_('Max',_1,_1) or similar due to tuple indexing
                parts = v.name.split('_')
                # Reconstruct parts to handle pulp's string representation of tuples
                barista_name = parts[1].strip("('").strip("',")
                day_index = int(parts[2].strip(","))
                block_index = int(parts[3].strip(")"))
                schedule[barista_name][day_index].append(block_index)

    # Pretty print the schedule
    for b in baristas:
        print(f"\nBarista: {b}")
        total_weekly_hours = 0
        for d_idx in days_indices:
            daily_hours = 0
            if schedule[b][d_idx]:
                blocks_worked = sorted(schedule[b][d_idx])
                blocks_str = ", ".join([Block_Name[k] for k in blocks_worked])
                daily_hours = len(blocks_worked) * H
                print(f"  {days[d_idx]}: {blocks_str} ({daily_hours} hours)")
            else:
                print(f"  {days[d_idx]}: No shifts")
            total_weekly_hours += daily_hours
        print(f"  Total Weekly Hours: {total_weekly_hours} (Min: {minWeekly_default[b]})\n")

    print(f"Total Hours Worked (Objective Value): {pl.value(prob.objective)} hours")

else:
    print("No optimal solution found.")

Status: Optimal

Optimal Schedule:

Barista: Max
  Monday: No shifts
  Tuesday: 19:00-23:00 (4 hours)
  Wednesday: 11:00-15:00 (4 hours)
  Thursday: No shifts
  Friday: No shifts
  Saturday: 15:00-19:00 (4 hours)
  Sunday: No shifts
  Total Weekly Hours: 12 (Min: 12)


Barista: Jiwa
  Monday: 19:00-23:00 (4 hours)
  Tuesday: 15:00-19:00 (4 hours)
  Wednesday: 07:00-11:00 (4 hours)
  Thursday: No shifts
  Friday: No shifts
  Saturday: 11:00-15:00 (4 hours)
  Sunday: No shifts
  Total Weekly Hours: 16 (Min: 12)


Barista: Fore
  Monday: No shifts
  Tuesday: No shifts
  Wednesday: 15:00-19:00 (4 hours)
  Thursday: 15:00-19:00 (4 hours)
  Friday: 11:00-15:00 (4 hours)
  Saturday: No shifts
  Sunday: No shifts
  Total Weekly Hours: 12 (Min: 12)


Barista: Donna
  Monday: 11:00-15:00 (4 hours)
  Tuesday: 07:00-11:00 (4 hours)
  Wednesday: No shifts
  Thursday: 07:00-11:00, 11:00-15:00 (8 hours)
  Friday: 07:00-11:00 (4 hours)
  Saturday: 07:00-11:00, 19:00-23:00 (8 hours)
  Sunday: 11:00-15: